In [19]:
"""
This script can be used as skelton code to read the challenge train and test
csvs, to train a trivial model, and write data to the submission file.
"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

In [12]:
from sklearn.metrics import accuracy_score

## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [13]:
## Handle missing values
train_df.fillna('NA', inplace=True)
test_df.fillna('NA', inplace=True)

In [14]:
## Filtering column "mail_type"
train_x = train_df[['mail_type']]
train_y = train_df[['label']]

test_x = test_df[['mail_type']]

print(test_x)

                   mail_type
Id                          
0      multipart/alternative
1      multipart/alternative
2      multipart/alternative
3                  text/html
4      multipart/alternative
...                      ...
10740  multipart/alternative
10741  multipart/alternative
10742              text/html
10743  multipart/alternative
10744              text/html

[10745 rows x 1 columns]


In [37]:
## Do one hot encoding of categorical feature
feat_enc = OneHotEncoder()
feat_enc.fit(train_x)
train_x_featurized = feat_enc.transform(train_x)
test_x_featurized = feat_enc.transform(test_x)

# print(test_x_featurized.data)
# data = np.array(train_df['images'])
# print(type(data))
# train_X = np.hstack((train_x_featurized.data, data))
# print(train_X)
train_X = train_df[['images']]
test_X = test_df[['images']]


       images
Id           
0          23
1           1
2           0
3           4
4           4
...       ...
25061       6
25062       3
25063       4
25064       0
25065      20

[25066 rows x 1 columns]
       images
Id           
0           1
1           0
2           1
3          13
4           3
...       ...
10740       4
10741       0
10742      20
10743       4
10744       0

[10745 rows x 1 columns]


In [38]:
## Train a simple KNN classifier using featurized data
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X, train_y)
pred_y = neigh.predict(test_X)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')

